In [1]:
import pandas as pd
import numpy as np
from slugify import slugify
pd.set_option('display.max_columns',None)
from address_extractor import get_detailed_address

In [2]:
df_full = pd.read_csv(r'./nipin_homefairbd.csv')

In [3]:
df1 = df_full.copy()

In [4]:
df1.head(3)

,title,building_type,area,num_bed,num_bath,location,price,amenities,purpose,listing_url
0,Apartment For Sale in Mirpur Ahmednagar @ 1350...,Apartment,1350 sqft,3 Bed(s),3 Bath(s),"Mirpur-1, Dhaka",BDT 7000000,"Balcony,Drawing,Dining,Electricity,Water,Lift,...",Sale,https://homefairbd.com/ad/apartment-for-sale-i...
1,Used Apartment Sale in Uttara @ 1200 Sqft,Apartment,1200 sqft,3 Bed(s),3 Bath(s),Dhaka,BDT 7500000,"balcony,Drawing,Dining,Electricity,Water,Lift,...",Sale,https://homefairbd.com/ad/apartment-sale-in-ut...
2,South facing used apartment sale in Mogbazar@1...,Apartment,1200 sqft,3 Bed(s),3 Bath(s),Dhaka,BDT 9000000,"balcony,Drawing,Dining,Electricity,Water,Lift,...",sale,https://homefairbd.com/ad/south-facing-used-ap...


In [5]:
df1.shape

(2787, 10)

In [6]:
indexvalue = []
for i in df1.index:
    #print(df1.loc[i,'num_bed'])
    if (df1.loc[i,'num_bed'] is not np.NAN) and ('Bed' not in df1.loc[i,'num_bed']):
        indexvalue.append(i)
    elif (df1.loc[i,'num_bath'] is not np.NAN) and ('Bath' not in df1.loc[i,'num_bath']):
        indexvalue.append(i)
    elif (df1.loc[i,'area'] is not np.NAN) and ('sqft' not in df1.loc[i,'area']) and ('Comspacesale' not in df1.loc[i,'building_type']) :
        indexvalue.append(i)
    elif (len(df1.loc[i,'area'].split())<2) or (len(df1.loc[i,'area'].split())>2):
        indexvalue.append(i)
len(indexvalue)

412

In [7]:
df2 = df1.iloc[indexvalue,:]

In [8]:
df2.to_csv('./errors_raw_data.csv')

In [9]:
df3 = df1.iloc[~df1.index.isin(indexvalue),:]
#df3.reset_index(drop=True)

In [10]:
df3.shape

(2375, 10)

In [11]:
df_fin = pd.DataFrame()

In [12]:
df4 = df3.copy()

In [13]:
"""
    Loop through `area` column, while:
        - converting `Katha` value to `sqft` value
        - removing the unit in the value, to only have the number left
"""

for index, row in df4.iterrows(): # loop through each sample
    
    # The code may take time, log in the console to keep track of things
    if index==0 or index%1000==0:
        print(f"Currently processing sample {index}...")
        
    # retrieve the area
    sample_area = df4.loc[index, "area"] 
    splitted_sample_area = sample_area.split()
    #print(splitted_sample_area, len(splitted_sample_area))
    # making sure there is only the value and the unit in sample_area
    if len(splitted_sample_area)>2 or len(splitted_sample_area)<2:
        print(f"Sample of index {index} has a suspicious value as area: {sample_area}")
        splitted_sample_area.insert(1,splitted_sample_area[0])
        splitted_sample_area[0] = 1 #defaulting to 1
        #break
        
    area = float(str(splitted_sample_area[0]).replace(" ","") ) # will contain the area; eg: 1345
    area_unit = splitted_sample_area[1].lower() # will contain the unit; eg: sqft
    
    # making sure all units are taken into account
    if area_unit not in ["sqft","katha","sothok","bigha","akor"]:
        print(f"Sample of index {index} has a unit not taken into account for its area: {sample_area}")
        break
   
    # converting katha area to sqft area (1 Katha = 720 sqft => Thanks @Kausthab Dutta Phukan )
    if area_unit=="katha":
        area *= 720
        
    # updating the area of the sample in the dataframe
    df4.loc[index, "area"] = area
    
print("Processing has come to an end")

# Converting area to decimal
df4["area"] = df4["area"].astype(float)

Currently processing sample 0...
Currently processing sample 1000...
Currently processing sample 2000...
Processing has come to an end


In [14]:
df4.head(2)

,title,building_type,area,num_bed,num_bath,location,price,amenities,purpose,listing_url
0,Apartment For Sale in Mirpur Ahmednagar @ 1350...,Apartment,1350.0,3 Bed(s),3 Bath(s),"Mirpur-1, Dhaka",BDT 7000000,"Balcony,Drawing,Dining,Electricity,Water,Lift,...",Sale,https://homefairbd.com/ad/apartment-for-sale-i...
1,Used Apartment Sale in Uttara @ 1200 Sqft,Apartment,1200.0,3 Bed(s),3 Bath(s),Dhaka,BDT 7500000,"balcony,Drawing,Dining,Electricity,Water,Lift,...",Sale,https://homefairbd.com/ad/apartment-sale-in-ut...


In [15]:
df4["city"] = np.NaN
df4["locality"] = np.NaN
df4["address"] = np.nan

In [16]:
df4.head(4)

,title,building_type,area,num_bed,num_bath,location,price,amenities,purpose,listing_url,city,locality,address
0,Apartment For Sale in Mirpur Ahmednagar @ 1350...,Apartment,1350.0,3 Bed(s),3 Bath(s),"Mirpur-1, Dhaka",BDT 7000000,"Balcony,Drawing,Dining,Electricity,Water,Lift,...",Sale,https://homefairbd.com/ad/apartment-for-sale-i...,NaN,NaN,NaN
1,Used Apartment Sale in Uttara @ 1200 Sqft,Apartment,1200.0,3 Bed(s),3 Bath(s),Dhaka,BDT 7500000,"balcony,Drawing,Dining,Electricity,Water,Lift,...",Sale,https://homefairbd.com/ad/apartment-sale-in-ut...,NaN,NaN,NaN
2,South facing used apartment sale in Mogbazar@1...,Apartment,1200.0,3 Bed(s),3 Bath(s),Dhaka,BDT 9000000,"balcony,Drawing,Dining,Electricity,Water,Lift,...",sale,https://homefairbd.com/ad/south-facing-used-ap...,NaN,NaN,NaN
3,সিপাহীবাগের শাহেরুনবাগে ১০৭৪ স্কয়ার ফিটের ফ্...,Apartment,1074.0,3 Bed(s),2 Bath(s),"Khilgoan, Dhaka",BDT 5700000,1 Balcony,NaN,https://homefairbd.com/ad/%E0%A6%B8%E0%A6%BF%E...,NaN,NaN,NaN


In [17]:
# New code

"""
    Loop through `location` column, while splitting each location to city, zone, address and add them
        to the relevant column
"""

for index, row in df4.iterrows(): # loop through each sample
    
    # The code may take time, log in the console to keep track of things
    if index==0 or index%1000==0:
        print(f"Currently processing sample {index}...")
        
    # retrieve the location
    location = df4.loc[index, "location"]
    
    # split the location to dictionary with Area, City, Address as keys
    location_dict = get_detailed_address(location)
    #print(location_dict)
    
    city = location_dict.get("city", np.NaN)
    locality = location_dict.get("area", np.NaN)
    address = location_dict.get("address", np.NaN)
    

    # updating the relevant columns of the sample in the dataframe
    df4.loc[index, "city"] = city
    df4.loc[index, "locality"] = locality
    df4.loc[index, "address"] = address

print("Processing has come to an end")

Currently processing sample 0...
Currently processing sample 1000...
Currently processing sample 2000...
Processing has come to an end


In [18]:
df4.head(4)

,title,building_type,area,num_bed,num_bath,location,price,amenities,purpose,listing_url,city,locality,address
0,Apartment For Sale in Mirpur Ahmednagar @ 1350...,Apartment,1350.0,3 Bed(s),3 Bath(s),"Mirpur-1, Dhaka",BDT 7000000,"Balcony,Drawing,Dining,Electricity,Water,Lift,...",Sale,https://homefairbd.com/ad/apartment-for-sale-i...,Dhaka,,Mirpur-1
1,Used Apartment Sale in Uttara @ 1200 Sqft,Apartment,1200.0,3 Bed(s),3 Bath(s),Dhaka,BDT 7500000,"balcony,Drawing,Dining,Electricity,Water,Lift,...",Sale,https://homefairbd.com/ad/apartment-sale-in-ut...,Dhaka,,
2,South facing used apartment sale in Mogbazar@1...,Apartment,1200.0,3 Bed(s),3 Bath(s),Dhaka,BDT 9000000,"balcony,Drawing,Dining,Electricity,Water,Lift,...",sale,https://homefairbd.com/ad/south-facing-used-ap...,Dhaka,,
3,সিপাহীবাগের শাহেরুনবাগে ১০৭৪ স্কয়ার ফিটের ফ্...,Apartment,1074.0,3 Bed(s),2 Bath(s),"Khilgoan, Dhaka",BDT 5700000,1 Balcony,NaN,https://homefairbd.com/ad/%E0%A6%B8%E0%A6%BF%E...,Dhaka,,Khilgoan


In [19]:
df_fin = df4.loc[:,['city','locality','address','area']]
df_fin

,city,locality,address,area
0,Dhaka,,Mirpur-1,1350.0
1,Dhaka,,,1200.0
2,Dhaka,,,1200.0
3,Dhaka,,Khilgoan,1074.0
4,Dhaka,,,1850.0
...,...,...,...,...
2782,Dhaka,,,1475.0
2783,Dhaka,,,2000.0
2784,Dhaka,,Dakkhin Khan,950.0
2785,Dhaka,,,1650.0


In [20]:
#building_type
df_fin['building_type']  = df4.building_type.replace(' ','')
df_fin.building_type.unique()

array([' Apartment ', ' Condos '], dtype=object)

In [21]:
#building_nature
df_fin['building_nature'] = df4.building_type
df_fin.building_nature = df_fin.building_nature.replace(' Apartment ','Residential')
df_fin.building_nature = df_fin.building_nature.replace(' Land ','Commercial')
df_fin.building_nature = df_fin.building_nature.replace(' Condos ','Commercial')
df_fin.building_nature = df_fin.building_nature.replace(' Comspacesale ','Commercial')
#df_fin.building_nature = df_fin.building_nature.replace(' Apartment ','Residential')
df_fin.building_nature.fillna('Commercial',inplace=True)
df_fin.building_nature.unique()

array(['Residential', 'Commercial'], dtype=object)

In [22]:
#num of bath and bed rooms
df_fin['num_bath_rooms'] = df4.num_bath.apply(lambda x: x.split(' Bath(s)')[0])
df_fin['num_bed_rooms'] = df4.num_bed.apply(lambda x: x.split(' Bed(s)')[0])

#fill na with 0
df_fin.num_bath_rooms.fillna(0,inplace=True)
df_fin.num_bed_rooms.fillna(0,inplace=True)

df_fin.num_bath_rooms.replace(' ','')
df_fin.num_bed_rooms.replace(' ','')

df_fin.num_bath_rooms.isna().sum(), df_fin.num_bed_rooms.isna().sum()

(0, 0)

In [23]:
# price
df_fin['price'] = df4.price.apply(lambda x: str(x).split('BDT')[-1] )
df_fin.price = df_fin.price.astype(float)
df_fin.price.dtype

dtype('float64')

In [24]:
#purpose
#since the data is not available, and the website mentions the sale of property, defaulting value to sale
df_fin['purpose'] = df4.purpose

In [25]:
df_fin['amenities'] = df4.amenities

In [26]:
aminities = df_fin.amenities.unique().tolist()

In [27]:
aminity = []

for i in aminities:
    j = i.split(',')
    #print("J:",j)
    for k in j:
        l = str(k.lower())
        l = l.replace('[','')
        l = l.replace(']','')
        l = l.replace("'",'')
        l = l.replace(" ",'')
        #print("K:",l)
        if (l not in aminity) and (len(k.split())<2):
            #print("L1: ",l )
            aminity.append(l)
        elif (l not in aminity) and (len(k.split())>1):
            m = k.split()[1].lower()
            if m not in aminity:
                #print(m)
                #print("L2: ",m, l,str(k.split()[1].lower()))
                aminity.append(str(m))
            
        print("Aminity :",aminity)

Aminity : ['balcony']
Aminity : ['balcony', 'drawing']
Aminity : ['balcony', 'drawing', 'dining']
Aminity : ['balcony', 'drawing', 'dining', 'electricity']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking']
Amini

Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator']
Aminity : 

Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'din

Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'din

Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'din

Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'dining', 'electricity', 'water', 'lift', 'parking', 'generator', 'none']
Aminity : ['balcony', 'drawing', 'din

In [28]:
aminity

['balcony',
 'drawing',
 'dining',
 'electricity',
 'water',
 'lift',
 'parking',
 'generator',
 'none']

In [29]:
"""
    Loop through `amenities` column, while:
         * Converting the dictionnaries keys to new columns; the values of the keys are becoming
             the new columns values for the corresponding sample
"""

for index, row in df_fin.iterrows(): # loop through each sample
    
    # The code may take time, log in the console to keep track of things
    if index==0 or index%1000==0:
        print(f"Currently processing sample {index}...")
        
    # If current sample doen't have amenities, go to the next one
    if pd.isna(df_fin.loc[index, "amenities"]):
        continue
    
    # retrieve the amenities
    sample_amenities = str(df_fin.loc[index, "amenities"]).replace("'","\"")
    sample_amenities = (sample_amenities.replace(' ','_'))
    #print(sample_amenities)
    
    #amenities_dict = eval(sample_amenities)
    
    # Go through each key in the amenities dictionnary
    for key in sample_amenities.split(','):
        
        for k in key:
            l = str(k.lower())
            l = l.replace('[','')
            l = l.replace(']','')
            l = l.replace("'",'')
            l = l.replace(" ",'')
            #print("K:",l)
            if (l not in aminity) and (len(k.split())<2):
                #print("L1: ",l )
                aminity.append(l)
            elif (l not in aminity) and (len(k.split())>1):
                m = k.split()[1].lower()
                if m not in aminity:
                    #print(m)
                    #print("L2: ",m, l,str(k.split()[1].lower()))
                    aminity.append(str(m))
        
        # put a suffix to the new column name, so that collaborators know it was generated from amenities feature
        for i in aminity:
            column_name = slugify(i)+"-amenity"
        #print(column_name)
        
        # Create new column based on the key if not already existing
        if column_name not in df_fin.columns.to_list():
            df_fin[column_name]= np.NaN # Giving NaN as the default value for the column
        
        # Affecting to the new column created, for the current sample, the value of the dictionary's key
        df_fin.loc[index, column_name] = 'Yes'
        

Currently processing sample 0...
Currently processing sample 1000...
Currently processing sample 2000...


In [30]:
aminity

['balcony',
 'drawing',
 'dining',
 'electricity',
 'water',
 'lift',
 'parking',
 'generator',
 'none',
 'b',
 'a',
 'l',
 'c',
 'o',
 'n',
 'y',
 'd',
 'r',
 'w',
 'i',
 'g',
 'e',
 't',
 'f',
 '_',
 '2',
 'p',
 'k',
 '3',
 '1',
 '4',
 '5',
 '6',
 '',
 '"',
 '8',
 '0',
 '-',
 '7']

In [31]:
df_fin

,city,locality,address,area,building_type,building_nature,num_bath_rooms,num_bed_rooms,price,purpose,amenities,y-amenity,g-amenity,t-amenity,f-amenity,2-amenity,k-amenity,3-amenity,1-amenity,4-amenity,5-amenity,6-amenity,-amenity,8-amenity,0-amenity,7-amenity
0,Dhaka,,Mirpur-1,1350.0,Apartment,Residential,3,3,7000000.0,Sale,"Balcony,Drawing,Dining,Electricity,Water,Lift,...",Yes,Yes,Yes,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Dhaka,,,1200.0,Apartment,Residential,3,3,7500000.0,Sale,"balcony,Drawing,Dining,Electricity,Water,Lift,...",NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Dhaka,,,1200.0,Apartment,Residential,3,3,9000000.0,sale,"balcony,Drawing,Dining,Electricity,Water,Lift,...",NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dhaka,,Khilgoan,1074.0,Apartment,Residential,2,3,5700000.0,NaN,1 Balcony,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Dhaka,,,1850.0,Apartment,Residential,4,3,13875000.0,sale,"balcony,Drawing,Dining,Electricity,Water,Lift,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2782,Dhaka,,,1475.0,Apartment,Residential,3,3,10325000.0,NaN,3 Balcony,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes
2783,Dhaka,,,2000.0,Apartment,Residential,4,4,9200000.0,Sale,"balcony,Drawing,dining,Electricity,Water,Lift,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes
2784,Dhaka,,Dakkhin Khan,950.0,Apartment,Residential,2,2,4100000.0,Sale,"Balcony,Drawing,Dining,Parking,Electricity,Wat...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes
2785,Dhaka,,,1650.0,Apartment,Residential,3,3,12375000.0,NaN,2 Balcony,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes


In [32]:
df_fin.columns

Index(['city', 'locality', 'address', 'area', 'building_type',
       'building_nature', 'num_bath_rooms', 'num_bed_rooms', 'price',
       'purpose', 'amenities', 'y-amenity', 'g-amenity', 't-amenity',
       'f-amenity', '2-amenity', 'k-amenity', '3-amenity', '1-amenity',
       '4-amenity', '5-amenity', '6-amenity', '-amenity', '8-amenity',
       '0-amenity', '7-amenity'],
      dtype='object')

In [33]:
columns_drop = ['amenities','y-amenity','g-amenity', 't-amenity', 'f-amenity', '2-amenity', 'k-amenity',
                '3-amenity', '1-amenity', '4-amenity', '5-amenity', '6-amenity','-amenity', '8-amenity', '0-amenity', '7-amenity']

In [34]:
df_fin.drop(columns=columns_drop,inplace=True)

In [35]:
df_fin['property_description'] = df4.title
df_fin['property_overview'] = np.NaN

In [36]:
df_fin.head(6)

,city,locality,address,area,building_type,building_nature,num_bath_rooms,num_bed_rooms,price,purpose,property_description,property_overview
0,Dhaka,,Mirpur-1,1350.0,Apartment,Residential,3,3,7000000.0,Sale,Apartment For Sale in Mirpur Ahmednagar @ 1350...,NaN
1,Dhaka,,,1200.0,Apartment,Residential,3,3,7500000.0,Sale,Used Apartment Sale in Uttara @ 1200 Sqft,NaN
2,Dhaka,,,1200.0,Apartment,Residential,3,3,9000000.0,sale,South facing used apartment sale in Mogbazar@1...,NaN
3,Dhaka,,Khilgoan,1074.0,Apartment,Residential,2,3,5700000.0,NaN,সিপাহীবাগের শাহেরুনবাগে ১০৭৪ স্কয়ার ফিটের ফ্...,NaN
4,Dhaka,,,1850.0,Apartment,Residential,4,3,13875000.0,sale,"Semi ready apartment sale in Aftabnagar,Block-...",NaN
5,Dhaka,,,1650.0,Apartment,Residential,4,4,7425000.0,NaN,চন্দ্রিমা মডেল টাউনে ১৬৫০ স্কয়ার ফিটের ফ্ল্যাট,NaN


In [37]:
df_fin.shape

(2375, 12)

In [38]:
df_fin.head(4)

,city,locality,address,area,building_type,building_nature,num_bath_rooms,num_bed_rooms,price,purpose,property_description,property_overview
0,Dhaka,,Mirpur-1,1350.0,Apartment,Residential,3,3,7000000.0,Sale,Apartment For Sale in Mirpur Ahmednagar @ 1350...,NaN
1,Dhaka,,,1200.0,Apartment,Residential,3,3,7500000.0,Sale,Used Apartment Sale in Uttara @ 1200 Sqft,NaN
2,Dhaka,,,1200.0,Apartment,Residential,3,3,9000000.0,sale,South facing used apartment sale in Mogbazar@1...,NaN
3,Dhaka,,Khilgoan,1074.0,Apartment,Residential,2,3,5700000.0,NaN,সিপাহীবাগের শাহেরুনবাগে ১০৭৪ স্কয়ার ফিটের ফ্...,NaN


In [39]:
df_fin.to_csv(r'./cleaned_homefairbd_spider.csv',index=False)